# Generation Module Verification

This notebook tests the `src.generation` module which provides:
- **Query Rewriter** - Converts context-dependent questions to standalone form
- **Generator** - Produces answers with I_DONT_KNOW fallback
- **Retrieval Grader** - CRAG component for document relevance
- **Hallucination Grader** - Self-RAG component for answer grounding

**Note:** Full tests require GPU (Llama 3.1 8B). Run on Kaggle for complete testing.

In [1]:
import sys
import os

sys.path.append(os.path.abspath(".."))

from src.generation import (
    create_generation_components,
    GenerationComponents,
    GradeDocuments,
    GradeHallucinations
)

/home/marcantoniolopez/Documenti/github/projects/llm-semeval-task8/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step 1: Verify Pydantic Models

These define the JSON output schemas for graders.

In [2]:
print("📊 Pydantic Models:")
print(f"   • GradeDocuments: {list(GradeDocuments.model_fields.keys())}")
print(f"   • GradeHallucinations: {list(GradeHallucinations.model_fields.keys())}")

# Test instantiation
doc_grade = GradeDocuments(binary_score="yes")
hal_grade = GradeHallucinations(binary_score="no")

print(f"\n   Example GradeDocuments: {doc_grade.model_dump()}")
print(f"   Example GradeHallucinations: {hal_grade.model_dump()}")

print("\n✅ Pydantic models work!")

📊 Pydantic Models:
   • GradeDocuments: ['binary_score']
   • GradeHallucinations: ['binary_score']

   Example GradeDocuments: {'binary_score': 'yes'}
   Example GradeHallucinations: {'binary_score': 'no'}

✅ Pydantic models work!


## Step 2: Verify GenerationComponents Dataclass

In [3]:
import dataclasses

print("📊 GenerationComponents:")
if dataclasses.is_dataclass(GenerationComponents):
    fields = [f.name for f in dataclasses.fields(GenerationComponents)]
    print(f"   • Type: dataclass")
    print(f"   • Fields: {fields}")
else:
    print(f"   • Type: {type(GenerationComponents).__name__}")

print("\n✅ GenerationComponents structure verified!")

📊 GenerationComponents:
   • Type: dataclass
   • Fields: ['llm', 'query_rewriter', 'generator', 'retrieval_grader', 'hallucination_grader']

✅ GenerationComponents structure verified!


## Step 3: Test Factory Function (GPU Required)

**Uncomment and run on Kaggle with GPU.**

In [ ]:
components = create_generation_components()

print(f"\n📊 Components Created:")
print(f"   • llm: {type(components.llm).__name__}")
print(f"   • query_rewriter: {type(components.query_rewriter).__name__}")
print(f"   • generator: {type(components.generator).__name__}")
print(f"   • retrieval_grader: {type(components.retrieval_grader).__name__}")
print(f"   • hallucination_grader: {type(components.hallucination_grader).__name__}")

print("\n✅ All components initialized!")

print("⏭️ Skipped (requires GPU). Uncomment on Kaggle.")

Creating Generation Components with model: meta-llama/Meta-Llama-3.1-8B-Instruct...


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

## Step 4: Test Chains (GPU Required)

**Uncomment and run on Kaggle with GPU.**

In [ ]:
# Test Query Rewriter
print("🔄 Testing Query Rewriter...")
result = components.query_rewriter.invoke({
    "chat_history": "Human: Who is the CEO?\nAI: Tim Cook.",
    "question": "How old is he?"
})
print(f"   Input: 'How old is he?' (with context)")
print(f"   Output: {result}")

# Test Generator
print("\n🔄 Testing Generator...")
result = components.generator.invoke({
    "context": "Tim Cook is the CEO of Apple. He was born in 1960.",
    "question": "How old is Tim Cook?"
})
print(f"   Question: 'How old is Tim Cook?'")
print(f"   Answer: {result}")

print("\n✅ All chains work!")

print("⏭️ Skipped (requires GPU). Uncomment on Kaggle.")

## Summary

**Local verification (without GPU):**
- ✅ Imports work
- ✅ Pydantic models work
- ✅ GenerationComponents structure verified

**Requires Kaggle GPU:**
- ⏳ create_generation_components()
- ⏳ Query Rewriter chain
- ⏳ Generator chain
- ⏳ Retrieval Grader chain
- ⏳ Hallucination Grader chain